In [51]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Mon Apr 22 16:46:25 2024

@author: maialen

Script to generate an interactive plot to show the top 5 cell types obtained
from an spatial deconvolution analysis

Input: 
    => CSV file containing normalized weights [barcodes as rownames, cell types as colnames]
    => CSV file containing spatial coordinates [barcodes as rownames, x and y coordinates as columns]

"""
# <! ------------------------------------------------------------------------!>
# <!                       IMPORTS                                           !>
# <! ------------------------------------------------------------------------!>
import pandas as pd
from math import pi
from bokeh.plotting import figure, output_file, save,output_notebook, show, curdoc

from bokeh.transform import cumsum
from bokeh.models import ColumnDataSource, HoverTool,Range1d
from bokeh.palettes import Category20
from bokeh.models import CustomJS, TapTool


In [45]:
# output_notebook()

# modularized code 

In [26]:
    
    # <! ------------------------------------------------------------------------!>
    # <!                           DATA PREPARATION                              !>
    # <! ------------------------------------------------------------------------!>
    

def process_data(norm_weights_filepath, st_coords_filepath, clustering_data_path, n_largest_cell_types):
        # Read spatial deconvolution result CSV file
        norm_weights_df = pd.read_csv(norm_weights_filepath).set_index('Unnamed: 0')
        norm_weights_df.index.name = None
        # print(norm_weights_df.head())
        
        # Read spatial coordinates CSV file
        st_coords_df = pd.read_csv(st_coords_filepath).set_index('Unnamed: 0')
        st_coords_df.index.name = None
        #print(st_coords_df.head())

        # Merge coordinate df and cell weight df
        merged_df = pd.concat([st_coords_df, norm_weights_df], axis = 1)
        data_with_clusters = pd.read_csv(clustering_data_path)
        clusters_col =  pd.DataFrame(data_with_clusters["BayesSpace"]).set_index(data_with_clusters["Unnamed: 0"])
        merged_df["Cluster"] = clusters_col
        # print(merged_df)
        # It will be difficult to show the information of all 54 cell types when hovering
        # Thus, for each barcoded spot, retrieve the maximum 5 weights and create new columns
        # accordingly. Those 5 max columns will be the info shown in the hovertool
        
        max_weights = norm_weights_df.apply(lambda x: x.nlargest(n_largest_cell_types).index.values, axis=1)
        # print(max_weights)
        
        # Create df columns with max cell types
        cell_type_storage_arrays = list()
        cell_value_storage_arrays = list()
        
        # Extract cell types with largest weights
        for i in range(n_largest_cell_types):
            
            cell_type_storage_array = list()
            cell_value_storage_array = list()
            
            for barcode in max_weights.index:
                
                max_cell_types = max_weights.loc[barcode]
                max_cell_type = max_cell_types[i]
                max_cell_value = merged_df.loc[barcode, max_cell_types[i]]
                
                cell_type_storage_array.append(max_cell_type)
                cell_value_storage_array.append(max_cell_value)
        
        
            cell_type_storage_arrays.append(cell_type_storage_array)    
            cell_value_storage_arrays.append(cell_value_storage_array)     
        
        # print(len(cell_type_storage_arrays[0]))
        # Assign to new columns in the dataframe
        for i in range(n_largest_cell_types):
            merged_df[''.join(['Deconv_cell', str(i + 1)])] = cell_type_storage_arrays[i]
            merged_df[''.join(['Deconv_cell', str(i + 1), '_value'])] = cell_value_storage_arrays[i]
        
        # Since we only consider the top N cell types, we need to correct the weight
        # values so that the scatterpies account to the totality of the circle (sum of weights == 1)
        
        deconv_weight_columns = [f"Deconv_cell{i + 1}_value" for i in range(n_largest_cell_types)]
        
        # Create new normalized columns
        for i in range(n_largest_cell_types):
            
            # Calculate the sum of the top cell type weights
            total = merged_df.loc[:, deconv_weight_columns].sum(axis=1)
            
            # Create column with corrected weight values
            merged_df[''.join(['Deconv_cell', str(i + 1), '_norm_value'])] =  merged_df[''.join(['Deconv_cell', str(i + 1), '_value'])] / total
            
        
        # SLim down the df by selecting columns of interest only
        columns_of_interest = ['x', 'y','Cluster' ] + [f"Deconv_cell{i + 1}_norm_value" for i in range(n_largest_cell_types)] \
            + [f"Deconv_cell{i + 1}" for i in range(n_largest_cell_types)]
        reduced_df = merged_df.loc[:, columns_of_interest]
        print(n_largest_cell_types)
        return reduced_df


In [27]:
norm_weights_filepath = "norm_celltype_weights_UKF242_T_ST.csv"
st_coords_filepath = "spatial_coords_UKF242_T_ST.csv"
n_largest_cell_types = 5
data_with_clustering = "seurat_metadata_UKF242_T_ST.csv"
processed_data = process_data(norm_weights_filepath, st_coords_filepath, data_with_clustering, n_largest_cell_types)

5


In [28]:
processed_data

,x,y,Cluster,Deconv_cell1_norm_value,Deconv_cell2_norm_value,Deconv_cell3_norm_value,Deconv_cell4_norm_value,Deconv_cell5_norm_value,Deconv_cell1,Deconv_cell2,Deconv_cell3,Deconv_cell4,Deconv_cell5
AAACAACGAATAGTTC-1,45.410158,118.725590,1,0.424571,0.213648,0.172143,0.116293,0.073345,Endo.capilar,OPC,Astrocyte,AC.like,TAM.MG.pro.infl.II
AAACAAGTATCTCCCA-1,379.248060,441.943374,6,0.468574,0.318053,0.096958,0.062711,0.053704,AC.like,Astrocyte,TAM.MG.aging.sig,Scavenging.endothelial,OPC
AAACAATCTACTAGCA-1,66.284182,221.118172,4,0.337220,0.336689,0.184351,0.071828,0.069911,AC.like,Astrocyte,Oligodendrocyte,OPC,Mono.anti.infl
AAACACCAATAACTGC-1,435.498062,126.049809,2,0.505964,0.318232,0.066054,0.061978,0.047773,AC.like,Astrocyte,Oligodendrocyte,TAM.MG.prolif,NPC.like.OPC
AAACAGAGCGACTCCT-1,140.991216,414.038100,5,0.478051,0.330142,0.084174,0.057661,0.049972,Astrocyte,AC.like,Oligodendrocyte,TAM.MG.aging.sig,Scavenging.endothelial
...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTTGTGTGTCAAGA-1,252.685555,348.339856,3,0.382348,0.215194,0.169748,0.167838,0.064872,Astrocyte,OPC.like,AC.like,Oligodendrocyte,OPC
TTGTTTCACATCCAGG-1,429.785171,213.500984,2,0.363567,0.330417,0.129142,0.097253,0.079620,Astrocyte,AC.like,Oligodendrocyte,OPC,TAM.MG.pro.infl.I
TTGTTTCATTAGTCTA-1,442.529312,167.797857,2,0.454729,0.291361,0.086550,0.086144,0.081216,AC.like,Astrocyte,Oligodendrocyte,Tip.like,TAM.MG.aging.sig
TTGTTTCCATACAACT-1,343.286145,157.397466,2,0.396994,0.388074,0.078608,0.076122,0.060202,AC.like,Astrocyte,OPC.like.Prolif,Oligodendrocyte,OPC


In [29]:

# <! ------------------------------------------------------------------------!>
# <!                       BOKEH VISUALIZATION                               !>
# <! ------------------------------------------------------------------------!>
from bokeh.models import  Wedge

# Define color dictionary
colordict = {
    "AC.like": "#CCCCCC",
    "AC.like.Prolif": "#FF6600",
    "Astrocyte": "#00FFCC",
    "B.cell": "#F0E442",
    "CD4.INF": "#0066FF",
    "CD4.rest": "#FF00FF",
    "CD8.cytotoxic": "#00FF00",
    "CD8.EM": "#FF6666",
    "CD8.NK.sig": "#FFCC00",
    "cDC1": "#00FFFF",
    "cDC2": "#FF0066",
    "DC1": "#CCFF00",
    "DC2": "#0000FF",
    "DC3": "#FFCCCC",
    "Endo.arterial": "#CC00FF",
    "Endo.capilar": "#66FF00",
    "Mast": "#FF00CC",
    "MES.like.hypoxia.independent": "#00CCFF",
    "MES.like.hypoxia.MHC": "#003399",
    "Mono.anti.infl": "#FF3366",
    "Mono.hypoxia": "#00FF66",
    "Mono.naive": "#FF9999",
    "Neuron": "#6600FF",
    "NK": "#FFE6E6",
    "NPC.like.neural": "#0072B2",
    "NPC.like.OPC": "#FF0000",
    "NPC.like.Prolif": "#999900",
    "Oligodendrocyte": "#666666",
    "OPC": "#CCFF99",
    "OPC.like": "#000000",
    "OPC.like.Prolif": "#990000",
    "pDC": "#993300",
    "Pericyte": "#996600",
    "Perivascular.fibroblast": "#999999",
    "Plasma.B": "#669900",
    "Prolif.T": "#339900",
    "Reg.T": "#CC79A7",
    "RG": "#009933",
    "Scavenging.endothelial": "#990099",
    "Scavenging.pericyte": "#009900",
    "SMC": "#330099",
    "SMC.COL": "#CC9999",
    "SMC.prolif": "#009999",
    "Stress.sig": "#990066",
    "TAM.BDM.anti.infl": "#990033",
    "TAM.BDM.hypoxia.MES": "#CC3333",
    "TAM.BDM.INF": "#CC6666",
    "TAM.BDM.MHC": "#660099",
    "TAM.MG.aging.sig": "#CCCC99",
    "TAM.MG.pro.infl.I": "#56B4E9",
    "TAM.MG.pro.infl.II": "#333333",
    "TAM.MG.prolif": "#99CC99",
    "Tip.like": "#99CC66",
    "VLMC": "#99CC33"
}
def vis_with_proportions(reduced_df, nb_spots_samples, show_legend = False, width =1000, height = 1000 , show_fig = False):        
        # Smaller sample
        test_df = reduced_df.iloc[1:nb_spots_samples, ].copy()
        # Create a single tooltip column for each circle
        test_df['tooltip_data'] = test_df.apply(lambda row: '<br>'. \
                                                      join([f"<span style='color: blue;'>{row[f'Deconv_cell{i+1}']}</span>: {row[f'Deconv_cell{i+1}_norm_value']*100:.2f}%" \
                                                            for i in range(n_largest_cell_types)] \
                                                           + [f"<span style='color: red;'> Spot</span>{ row['x'], row['y'] }"] ),\
                                                      axis=1)
        # Update the data dictionary
        data = {
            'x': [y / 100 for y in test_df.y.tolist()],
            'y': [-x / 100 for x in test_df.x.tolist()],
            'x_full': test_df.y.tolist(),
            'y_full': [-x for x in test_df.x.tolist()],
            'tooltip_data': test_df['tooltip_data'].tolist(),
        }
        
        for i in range(1, n_largest_cell_types + 1):
            data[f'DeconvCell{i}'] = test_df[f'Deconv_cell{i}'].tolist()
            data[f'DeconvCell{i}_w'] = test_df[f'Deconv_cell{i}_norm_value'].tolist()
        # Convert dictionary to dataframe
        df = pd.DataFrame(data)
        # Convert dataframe to a ColumnDataSource
        # Initialize the Bokeh plot
        p = figure(width=width, height=height,
                   title="Deconvolution results",
                   x_axis_label='x',
                   y_axis_label='y',
                   output_backend="webgl",
                  )
        # Create a Div for displaying the message
        for index, row in df.iterrows():
            x, y = row['x'], row['y']
            categories = row[[f'DeconvCell{i+1}_w' for i in range(n_largest_cell_types)]].values
            cell_types = row[[f'DeconvCell{i+1}' for i in range(n_largest_cell_types)]].values
            colors = tuple([colordict[x] for x in cell_types])
            # Create a single ColumnDataSource for all wedges in this circle
            circle_source = ColumnDataSource({
                'x': [x],
                'y': [y],
                'tooltip_data': [row['tooltip_data']]
            })
            start_angle = 0
            for i, category_value in enumerate(categories):
                end_angle = start_angle + category_value * 2 * pi
                wedge = p.wedge(x='x', y='y', radius=0.05, 
                        start_angle=start_angle, end_angle=end_angle,
                        line_color="white", fill_color=colors[i], 
                        legend_label=f"DeconvCell {i+1}", source=circle_source)
                
                start_angle = end_angle
        # # Show no legend
        p.legend.visible= show_legend
        hover = HoverTool(tooltips="""
            <div style="width:200px">
                <h3>Proportions:</h3>
                @tooltip_data
            </div>
        """)
        # Add the hover tool to the plot
        p.add_tools(hover)
        if show_fig :
            show(p)
        return p

In [31]:
nb_spots_samples = 50
vis_with_proportions(processed_data, nb_spots_samples, show_fig=True)


figure(id='p291456', ...)

In [34]:

# <! ------------------------------------------------------------------------!>
# <!                       BOKEH VISUALIZATION                               !>
# <! ------------------------------------------------------------------------!>

def vis_type_majoritaires(reduced_df, nb_spots_samples, show_figure = False, show_legend = False, width =1000, height = 1000 ):    
        # Smaller df for testing
        test_df = reduced_df.iloc[1:nb_spots_samples, ].copy()
        # Create a single tooltip column for each circle
        test_df['tooltip_data'] = test_df.apply(lambda row: '<br>'. \
                                                      join([f"<span style='color: blue;'>{row[f'Deconv_cell{1}']}</span>: {row[f'Deconv_cell{1}_norm_value']*100:.2f}%"] \
                                                           + [f"<span style='color: red;'> Spot</span>{ row['x'], row['y'] }"] ),\
                                                      axis=1)
        # Update the data dictionary
        data = {
            'x': [y / 100 for y in test_df.y.tolist()],
            'y': [-x / 100 for x in test_df.x.tolist()],
            'x_full': test_df.y.tolist(),
            'y_full': [-x for x in test_df.x.tolist()],
            'tooltip_data': test_df['tooltip_data'].tolist(),
        }
        data[f'DeconvCell{1}'] = test_df[f'Deconv_cell{1}'].tolist()
        data[f'DeconvCell{1}_w'] = test_df[f'Deconv_cell{1}_norm_value'].tolist()
        # Convert dictionary to dataframe
        df = pd.DataFrame(data)
        # Initialize the Bokeh plot
        p = figure(width =1000, height = 1000,
                    title = "Deconvolution results", 
                    x_axis_label = 'x',
                    y_axis_label = 'y', 
                   output_backend="webgl"
                    )
        for index, row in df.iterrows():
            x, y = row['x'], row['y']
            categorie = row[f'DeconvCell{1}_w' ]
            cell_type = row[f'DeconvCell{1}']
            color = colordict[cell_type] 
            # Create a single ColumnDataSource for all wedges in this circle
            circle_source = ColumnDataSource({
                'x': [x],
                'y': [y],
                'tooltip_data': [row['tooltip_data']]
            })
            wedge = p.wedge(x='x', y='y', radius=0.05, 
                    start_angle=0, end_angle=2*pi,
                    line_color="white", fill_color=color,  line_width = 0, 
                    source=circle_source)
            # p.scatter(x, y, size=5, marker="circle", fill_color=color, line_color=color, fill_alpha=0.6, line_width=2)
        hover = HoverTool(tooltips="""
            <div style="width:200px">
                <h3>Proportions:</h3>
                @tooltip_data
            </div>
        """)
        # Add the hover tool to the plot
        p.add_tools(hover)
        if show_figure:
            show(p)

In [36]:
nb_spots_samples = 1000
vis_type_majoritaires(processed_data, nb_spots_samples, show_figure=True)


In [39]:

# <! ------------------------------------------------------------------------!>
# <!                       BOKEH VISUALIZATION                               !>
# <! ------------------------------------------------------------------------!>

# Define color dictionary
clusters_colordict = {
    0: "#CCCCCC",
    1: "#FF6600",
    2: "#00FFCC",
    3: "#F0E442",
    4: "#0066FF",
    5: "#FF00FF",
    6: "#00FF00",
    7: "#FF6666",
    8: "#FFCC00",
    9: "#00FFFF",
    10: "#FF0066",
    11: "#CCFF00",
    12: "#0000FF",
    13: "#FFCCCC",
    14: "#CC00FF",
}
def vis_with_clustring(reduced_df, nb_spots_samples,  show_figure = False, show_legend = False, width =1000, height = 1000 ): 
        # Smaller df for testing
        test_df = reduced_df.iloc[1:nb_spots_samples, ].copy()
        # Create a single tooltip column for each circle
        test_df['tooltip_data'] = test_df.apply(lambda row: '<br>'.join( \
                                                [f"<span style='color: red;'> Spot</span> : (x = { row['x']:.2f}, y = {row['y']:.2f})"] \
                                                + [f"<span style='color: blue;'> Cluster</span> { row['Cluster']}"] ),\
                                                axis=1)
        # Update the data dictionary
        data = {
            'x': [y / 100 for y in test_df.y.tolist()],
            'y': [-x / 100 for x in test_df.x.tolist()],
            'tooltip_data': test_df['tooltip_data'].tolist(),
            'Cluster' : test_df['Cluster'].tolist()
        }
        # Convert dictionary to dataframe
        df = pd.DataFrame(data)
        # Initialize the Bokeh plot
        p = figure(width =width, height = height,
                    title = "Deconvolution results", 
                    x_axis_label = 'x',
                    y_axis_label = 'y', 
                   output_backend="webgl"
                    )
        for index, row in df.iterrows():
            x, y = row['x'], row['y']
            cluster = row['Cluster']
            color = clusters_colordict[cluster]
            # Create a single ColumnDataSource for all wedges in this circle
            circle_source = ColumnDataSource({
                'x': [x],
                'y': [y],
                'tooltip_data': [row['tooltip_data']]
            })
            
            wedge = p.wedge(x='x', y='y', radius=0.02, 
                    start_angle=0, end_angle=2*pi,
                    line_color="white", fill_color= color,  line_width = 0
                    , source=circle_source, legend_label= f"Cluster {cluster}")
        
        
        # Show no legend
        p.legend.visible= show_legend 
        hover = HoverTool(tooltips="""
            <div style="width:200px">
                @tooltip_data
            </div>
        """)
        # Add the hover tool to the plot
        p.add_tools(hover)
        # Configurer la légende
        p.legend.location = "top_right"
        p.legend.click_policy = "hide"
        if show_figure:
            show(p)

In [40]:
nb_spots_samples = processed_data.shape[0]
vis_with_clustring(processed_data, nb_spots_samples, show_figure = True)

In [41]:
processed_data.shape


(3875, 13)

In [49]:

# <! ------------------------------------------------------------------------!>
# <!                       BOKEH VISUALIZATION                               !>
# <! ------------------------------------------------------------------------!>

# Define color dictionary
clusters_colordict = {
    0: "#CCCCCC",
    1: "#FF6600",
    2: "#00FFCC",
    3: "#F0E442",
    4: "#0066FF",
    5: "#FF00FF",
    6: "#00FF00",
    7: "#FF6666",
    8: "#FFCC00",
    9: "#00FFFF",
    10: "#FF0066",
    11: "#CCFF00",
    12: "#0000FF",
    13: "#FFCCCC",
    14: "#CC00FF",
}
from bokeh.events import ButtonClick
from bokeh.models import BoxAnnotation, Label, Plot, Rect, Text, Button, CustomJS
from bokeh.plotting import figure
from bokeh.transform import factor_cmap
from bokeh.layouts import column, row, gridplot,Spacer

def vis_with_separate_clusters_view(reduced_df, nb_spots_samples, output,  show_legend = False, show_figure = False, width =1000, height = 1000 ): 
        # Smaller sample
        test_df = reduced_df.iloc[1:nb_spots_samples, ].copy()
        # Create a single tooltip column for each circle
        test_df['tooltip_data'] = test_df.apply(lambda row: '<br>'.join( \
                                                [f"<span style='color: red;'> Spot</span> : (x = { row['x']:.2f}, y = {row['y']:.2f})"] \
                                                + [f"<span style='color: blue;'> Cluster</span> { row['Cluster']}"] ),\
                                                axis=1)
        # Update the data dictionary
        data = {
            'x': [y / 100 for y in test_df.y.tolist()],
            'y': [-x / 100 for x in test_df.x.tolist()],
            'tooltip_data': test_df['tooltip_data'].tolist(),
            'Cluster' : test_df['Cluster'].tolist()
        }
        # Convert dictionary to dataframe
        df = pd.DataFrame(data)
        # Initialize the Bokeh plot
        p = figure(width =1000, height = 1000,
                    title = "Clustering results", 
                    x_axis_label = 'x',
                    y_axis_label = 'y', 
                   output_backend="webgl"
                    )
        
        # Créez un dictionnaire pour stocker les glyphs par cluster
        for index, row in df.iterrows():
            x, y = row['x'], row['y']
            cluster = row['Cluster']
            color = clusters_colordict[cluster]
            # Create a single ColumnDataSource for all wedges in this circle
            circle_source = ColumnDataSource({
                'x': [x],
                'y': [y],
                'tooltip_data': [row['tooltip_data']],
                'cluster' : [cluster]
            })
            
            wedge = p.wedge(x='x', y='y', radius=0.03, 
                    start_angle=0, end_angle=2*pi,
                    line_color="white", fill_color= color,  line_width = 0, legend_label=f"Cluster {cluster}"
                    , source=circle_source, name = f"Cluster_{cluster}", alpha=1, visible = True )
        # Create a single tooltip column for each circle
        test_df['tooltip_data'] = test_df.apply(lambda row: '<br>'. \
                                                      join([f"<span style='color: blue;'>{row[f'Deconv_cell{i+1}']}</span>: {row[f'Deconv_cell{i+1}_norm_value']*100:.2f}%" \
                                                            for i in range(n_largest_cell_types)] \
                                                           + [f"<span style='color: red;'> Spot</span>{ row['x'], row['y'] }"] ),\
                                                      axis=1)
        data["tooltip_data"] = test_df['tooltip_data'].tolist()
        for i in range(1, n_largest_cell_types + 1):
            data[f'DeconvCell{i}'] = test_df[f'Deconv_cell{i}'].tolist()
            data[f'DeconvCell{i}_w'] = test_df[f'Deconv_cell{i}_norm_value'].tolist()
        # Convert dictionary to dataframe
        df = pd.DataFrame(data)
        plot = figure(width=width, height=height,
                   title="Deconvolution results",
                   x_axis_label='x',
                   y_axis_label='y',
                   output_backend="webgl",
                  )
        # Create a Div for displaying the message
        for index, row in df.iterrows():
            x, y = row['x'], row['y']
            categories = row[[f'DeconvCell{i+1}_w' for i in range(n_largest_cell_types)]].values
            cell_types = row[[f'DeconvCell{i+1}' for i in range(n_largest_cell_types)]].values
            colors = tuple([colordict[x] for x in cell_types])
            # Create a single ColumnDataSource for all wedges in this circle
            circle_source = ColumnDataSource({
                'x': [x],
                'y': [y],
                'tooltip_data': [row['tooltip_data']]
            })
            start_angle = 0
            for i, category_value in enumerate(categories):
                end_angle = start_angle + category_value * 2 * pi
                wedge = plot.wedge(x='x', y='y', radius=0.05, 
                        start_angle=start_angle, end_angle=end_angle,
                        line_color="white", fill_color=colors[i], color = colors[i],
                        legend_label=f"Cluster {row['Cluster']}", source=circle_source, visible=False)
                start_angle = end_angle
        buttons = []
        button = Button(label=f"Show deconvolution by Cluster", width=100)
        button.js_on_click(CustomJS(args=dict(p = p, plot = plot), code="""
                p.visible = false;
                plot.visible = true;
            """))
        buttons.append(button)
        # Add a spacer between buttons
        spacer = Spacer(width=100)  # Adjust the width as needed
        buttons.append(spacer)
        # Add a "Show All" button
        show_all_button = Button(label="Show Clusters", width=100)
        show_all_button.js_on_click(CustomJS(args=dict(p = p, plot = plot), code="""
                p.visible = true;
                plot.visible = false;
        """))
        buttons.append(show_all_button)
        tap_tool = TapTool()
        p.add_tools(tap_tool)
        # Associez le callback au TapTool
        hover = HoverTool(tooltips="""
            <div style="width:200px">
                @tooltip_data
            </div>
        """)
        p.add_tools(hover)
        plot.add_tools(hover)
        leg = p.legend[0]
        p.add_layout(leg,'left')
        leg_plot = plot.legend[0]
        plot.add_layout(leg_plot,'left')
    
        from bokeh.layouts import column, row # to avoid a conflict with row from pandas
        # Create a layout with the buttons and the plot container
        layout = column(
            row(buttons),
            p,
            plot
        )

        p.legend.location = "top_right"
        p.legend.click_policy = "hide"
        p.legend.visible = True
        plot.legend.location = "top_right"
        plot.legend.click_policy = "hide"
        plot.visible = False
        plot.legend.visible = True

        if show_figure:
            show(layout)
        output_file(output)
        save(layout)



In [ ]:
nb_spots_samples = processed_data.shape[0]
output_html = "visium_plot_sample1.html"
vis_with_separate_clusters_view(processed_data, nb_spots_samples, output_html )